In [1]:
import pandas as pd
import sys
import os
from pathlib import Path
from config import DBConfig
import sqlite3

project_root = os.path.abspath('../../')
if project_root not in sys.path:
    sys.path.append(project_root)
    
from src.data.db import DBConnection, execute_query, get_pitcher_data

In [2]:
class NotebookDBConnection(DBConnection):
    def __init__(self):
        db_path = os.path.join(project_root, 'data', 'pitcher_stats.db')
        super().__init__(db_name=db_path)

In [3]:
pd.set_option('display.max_columns', 150)

In [16]:
with NotebookDBConnection() as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print([table[0] for table in tables])

['game_context', 'pitchers', 'sqlite_sequence', 'pitch_mix_features', 'team_stats', 'pitcher_ids', 'teams', 'games', 'statcast_pitches', 'game_stats', 'pitch_mix', 'team_batting_stats', 'batter_profiles', 'sequence_patterns', 'prediction_features', 'starter_probability']


In [17]:
with NotebookDBConnection() as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM teams LIMIT 10")
    teams = cursor.fetchall()
    print([team[1] for team in teams])

['LAD', 'NYY', 'BOS', 'CHC', 'SFG', 'NYM', 'HOU', 'ATL', 'PHI', 'OAK']


In [31]:
with NotebookDBConnection() as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM pitchers")
    game_context = cursor.fetchall()

In [28]:
df = pd.DataFrame(game_context)

In [29]:
df.head()

,0,1,2,3
0,1,"Sabathia, CC",sabathia cc,282332
1,2,"Pujols, Albert",pujols albert,405395
2,3,"Rodney, Fernando",rodney fernando,407845
3,4,"Pérez, Oliver",perez oliver,424144
4,5,"Mathis, Jeff",mathis jeff,425772
